# Librairies

In [61]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)


In [62]:
from pandas.core.reshape.encoding import DataFrame
data_path = "data/"
df = pd.read_csv(data_path + "joinedData_assessmentStudentAssessor3_clean Anonymized.csv")
df

,StudentId,AssessmentId,FirstAssessment_byStudent,A1,A2,A3,A4,A5,A6,A7,...,avgTime_allTaskData_grpAssessor,medTime_allTaskData_grpAssessor,avgTime_startAssessment_grpAssessor,medTime_startAssessment_grpAssessor,StudentAgeAtAssesment,AssessmentDuration,Log[DurationAssessment],assessmentDate,AssessmentStartTime,AssessmentEndTime
0,V1_0,V2_0,1,0,0,0,0,0,0,0,...,4:52 PM,5:21 PM,4:59:06 PM,6:08:14 PM,5.1,:0:00:11:51,6.566654,2/15/2023,8:25:59 PM,8:37:50 PM
1,V1_0,V2_122,0,0,0,0,0,0,0,0,...,4:52 PM,5:21 PM,4:59:06 PM,6:08:14 PM,5.2,:0:00:46:24,7.931755,4/15/2023,2:59:19 PM,3:45:43 PM
2,V1_1,V2_1,1,0,0,0,0,0,0,0,...,5:41 PM,5:41 PM,5:41:14 PM,5:41:14 PM,7.0,:0:00:00:58,4.051785,10/15/2023,5:41:14 PM,5:42:12 PM
3,V1_1,V2_572,0,0,0,0,0,0,0,0,...,2:28 PM,2:54 PM,3:49:28 PM,3:36:18 PM,7.0,:30:04:46:13,14.774544,10/15/2023,5:42:40 PM,10:28:53 PM
4,V1_10,V2_11,1,1,1,1,1,0,0,0,...,6:55 PM,8:31 PM,4:02:30 PM,1:31:34 PM,5.9,:0:03:01:24,9.295092,4/15/2023,3:47:01 PM,6:48:26 PM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23349,V1_998,V2_3546,0,2,2,4,2,4,4,2,...,12:36 PM,6:28 PM,1:44:17 PM,6:28:24 PM,4.4,:24:23:40:42,14.585083,3/17/2023,4:22:45 AM,4:03:27 AM
23350,V1_998,V2_5400,0,2,2,4,2,4,4,2,...,7:52 PM,11:24 PM,6:45:43 PM,4:23:26 PM,5.2,:0:00:28:55,7.459001,1/18/2023,11:07:22 PM,11:36:18 PM
23351,V1_998,V2_6886,0,2,2,4,2,4,4,2,...,4:24 PM,6:32 PM,5:42:43 PM,7:22:49 PM,5.8,:76:22:30:44,15.709743,7/18/2023,12:15:28 AM,10:46:12 PM
23352,V1_999,V2_1605,1,2,2,1,2,2,2,2,...,5:56 PM,6:00 PM,6:29:39 PM,6:35:07 PM,7.7,:0:00:51:33,8.036790,4/16/2023,5:22:08 PM,6:13:40 PM


# Cleaning data

In [63]:
print(len(df[df['IsMale'].isnull()])) # 374 NA gender
df['IsMale'] = np.where(df['IsMale'].isnull(), 3, df['IsMale']);
print(df.groupby('IsMale')['IsMale'].count())

374
IsMale
0.0     5139
1.0    17841
3.0      374
Name: IsMale, dtype: int64


In [64]:
df2 = df.iloc[:,3:547]
df2

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,...,Z19,Z20,Z21,Z22,Z23,Z24,Z25,Z26,Z27,Z28
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,2,2,2,...,0,1,1,1,0,0,1,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,...,0,0,1,1,1,1,1,0,1,0
4,1,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23349,2,2,4,2,4,4,2,1,1,1,...,0,1,1,0,0,0,0,0,0,0
23350,2,2,4,2,4,4,2,1,1,1,...,0,1,1,0,0,0,0,0,0,0
23351,2,2,4,2,4,4,2,1,2,2,...,1,1,1,1,1,1,1,1,1,1
23352,2,2,1,2,2,2,2,1,1,0,...,0,0,0,0,0,0,0,0,0,0


# Normalize Skills

In [65]:
ndf = df.copy()
scaler = MinMaxScaler()
ndf.iloc[:,3:547]=scaler.fit_transform(ndf.iloc[:,3:547].to_numpy())
print(ndf.iloc[:,3:547])

<ipython-input-65-e6fecb0886dc>:3: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  ndf.iloc[:,3:547]=scaler.fit_transform(ndf.iloc[:,3:547].to_numpy())


        A1   A2    A3   A4   A5   A6   A7   A8   A9  A10  ...  Z19  Z20  Z21  \
0      0.0  0.0  0.00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
1      0.0  0.0  0.00  0.0  0.0  0.0  0.0  1.0  1.0  1.0  ...  0.0  1.0  1.0   
2      0.0  0.0  0.00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
3      0.0  0.0  0.00  0.0  0.0  0.0  0.0  0.5  0.0  0.0  ...  0.0  0.0  1.0   
4      0.5  0.5  0.25  0.5  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
...    ...  ...   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
23349  1.0  1.0  1.00  1.0  1.0  1.0  1.0  0.5  0.5  0.5  ...  0.0  1.0  1.0   
23350  1.0  1.0  1.00  1.0  1.0  1.0  1.0  0.5  0.5  0.5  ...  0.0  1.0  1.0   
23351  1.0  1.0  1.00  1.0  1.0  1.0  1.0  0.5  1.0  1.0  ...  1.0  1.0  1.0   
23352  1.0  1.0  0.25  1.0  0.5  0.5  1.0  0.5  0.5  0.0  ...  0.0  0.0  0.0   
23353  1.0  1.0  0.25  1.0  1.0  1.0  1.0  0.5  0.5  0.0  ...  0.0  0.0  0.0   

       Z22  Z23  Z24  Z25  Z26  Z27  Z2

#

# Grouping skills

In [66]:
skills = ndf.iloc[:,3:547].columns
skills

Index(['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10',
       ...
       'Z19', 'Z20', 'Z21', 'Z22', 'Z23', 'Z24', 'Z25', 'Z26', 'Z27', 'Z28'],
      dtype='object', length=544)

In [67]:
i = 1
current = 'A'
for skill in skills:
    if(skill[0] == current):
      i = i + 1
      if(current =='Z' and i == 28):
        ndf[current] = ndf[[current+str(1), current+str(i)]].mean(axis=1)
    else :
      if(current == 'O'):
        current = chr(ord(current) + 1)
        i = 1
        continue 
      ndf[current] = ndf[[current+str(1), current+str(i - 1)]].mean(axis=1)
      i = 1
      current = chr(ord(current) + 1)
      

      

In [68]:
ndf

,StudentId,AssessmentId,FirstAssessment_byStudent,A1,A2,A3,A4,A5,A6,A7,...,Q,R,S,T,U,V,W,X,Y,Z
0,V1_0,V2_0,1,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.000,0.00,0.000,0.00,0.00,0.00,0.00,0.00,0.0,0.0
1,V1_0,V2_122,0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.000,0.00,1.000,0.00,0.00,0.50,0.00,0.00,0.0,0.5
2,V1_1,V2_1,1,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.000,0.00,0.000,0.00,0.00,0.00,0.00,0.00,0.0,0.0
3,V1_1,V2_572,0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.500,0.00,0.250,0.50,0.50,0.50,0.25,0.50,0.0,0.0
4,V1_10,V2_11,1,0.5,0.5,0.25,0.5,0.0,0.0,0.0,...,0.250,0.25,0.000,0.25,0.00,0.00,0.00,0.00,0.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23349,V1_998,V2_3546,0,1.0,1.0,1.00,1.0,1.0,1.0,1.0,...,0.500,0.50,0.500,0.00,0.75,0.75,0.75,0.50,0.5,0.5
23350,V1_998,V2_5400,0,1.0,1.0,1.00,1.0,1.0,1.0,1.0,...,0.500,0.50,0.500,0.00,0.75,0.75,0.75,0.75,0.5,0.5
23351,V1_998,V2_6886,0,1.0,1.0,1.00,1.0,1.0,1.0,1.0,...,0.500,0.50,0.875,0.00,0.75,1.00,1.00,1.00,1.0,1.0
23352,V1_999,V2_1605,1,1.0,1.0,0.25,1.0,0.5,0.5,1.0,...,0.125,0.00,0.250,0.00,0.75,0.50,0.25,1.00,0.5,0.5


# Weighted skills 

In [70]:
ndf_ = ndf.copy()
skills_ = ndf_.iloc[:,3:547].columns

factor = 1
for skill in skills_:
        ndf_[(skill +"_")] = ndf_[skill] + factor
        factor = factor + 0.01   
   

In [71]:
ndf_

,StudentId,AssessmentId,FirstAssessment_byStudent,A1,A2,A3,A4,A5,A6,A7,...,Z19_,Z20_,Z21_,Z22_,Z23_,Z24_,Z25_,Z26_,Z27_,Z28_
0,V1_0,V2_0,1,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,6.34,6.35,6.36,6.37,6.38,6.39,6.4,6.41,6.42,6.43
1,V1_0,V2_122,0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,6.34,7.35,7.36,7.37,6.38,6.39,7.4,6.41,6.42,7.43
2,V1_1,V2_1,1,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,6.34,6.35,6.36,6.37,6.38,6.39,6.4,6.41,6.42,6.43
3,V1_1,V2_572,0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,6.34,6.35,7.36,7.37,7.38,7.39,7.4,6.41,7.42,6.43
4,V1_10,V2_11,1,0.5,0.5,0.25,0.5,0.0,0.0,0.0,...,6.34,6.35,6.36,6.37,6.38,6.39,6.4,6.41,6.42,6.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23349,V1_998,V2_3546,0,1.0,1.0,1.00,1.0,1.0,1.0,1.0,...,6.34,7.35,7.36,6.37,6.38,6.39,6.4,6.41,6.42,6.43
23350,V1_998,V2_5400,0,1.0,1.0,1.00,1.0,1.0,1.0,1.0,...,6.34,7.35,7.36,6.37,6.38,6.39,6.4,6.41,6.42,6.43
23351,V1_998,V2_6886,0,1.0,1.0,1.00,1.0,1.0,1.0,1.0,...,7.34,7.35,7.36,7.37,7.38,7.39,7.4,7.41,7.42,7.43
23352,V1_999,V2_1605,1,1.0,1.0,0.25,1.0,0.5,0.5,1.0,...,6.34,6.35,6.36,6.37,6.38,6.39,6.4,6.41,6.42,6.43


In [72]:
factored_skills = ndf_.iloc[:,ndf_.columns.get_loc("A1_"):ndf_.columns.get_loc("Z28_")].columns
factored_skills

Index(['A1_', 'A2_', 'A3_', 'A4_', 'A5_', 'A6_', 'A7_', 'A8_', 'A9_', 'A10_',
       ...
       'Z18_', 'Z19_', 'Z20_', 'Z21_', 'Z22_', 'Z23_', 'Z24_', 'Z25_', 'Z26_',
       'Z27_'],
      dtype='object', length=543)

# Weighted categorized skills

In [77]:
ndf_["Basic_Skills"] = ndf_[["A1_", "H49_"]].mean(axis=1)
ndf_["Advanced_Skills"] = ndf_[["I1_", "Z28_"]].mean(axis=1)
print(ndf_["Basic_Skills"])
print(ndf_["Advanced_Skills"])

0        2.37
1        2.37
2        2.37
3        2.87
4        2.62
         ... 
23349    2.87
23350    3.37
23351    3.37
23352    2.87
23353    2.87
Name: Basic_Skills, Length: 23354, dtype: float64
0        5.090
1        5.590
2        5.090
3        5.590
4        5.590
         ...  
23349    5.590
23350    5.590
23351    6.090
23352    5.215
23353    5.340
Name: Advanced_Skills, Length: 23354, dtype: float64


# Normalize Skills

In [78]:
scaler = MinMaxScaler()
ndf_[["A1_", "Z28_"]]=scaler.fit_transform(ndf_[["A1_", "Z28_"]])
ndf_["Basic_Skills"]=scaler.fit_transform(ndf_[["Basic_Skills"]])
ndf_["Advanced_Skills"]=scaler.fit_transform(ndf_[["Advanced_Skills"]])
ndf_

,StudentId,AssessmentId,FirstAssessment_byStudent,A1,A2,A3,A4,A5,A6,A7,...,Z21_,Z22_,Z23_,Z24_,Z25_,Z26_,Z27_,Z28_,Basic_Skills,Advanced_Skills
0,V1_0,V2_0,1,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,6.36,6.37,6.38,6.39,6.4,6.41,6.42,0.0,0.00,0.000
1,V1_0,V2_122,0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,7.36,7.37,6.38,6.39,7.4,6.41,6.42,1.0,0.00,0.500
2,V1_1,V2_1,1,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,6.36,6.37,6.38,6.39,6.4,6.41,6.42,0.0,0.00,0.000
3,V1_1,V2_572,0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,7.36,7.37,7.38,7.39,7.4,6.41,7.42,0.0,0.50,0.500
4,V1_10,V2_11,1,0.5,0.5,0.25,0.5,0.0,0.0,0.0,...,6.36,6.37,6.38,6.39,6.4,6.41,6.42,0.0,0.25,0.500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23349,V1_998,V2_3546,0,1.0,1.0,1.00,1.0,1.0,1.0,1.0,...,7.36,6.37,6.38,6.39,6.4,6.41,6.42,0.0,0.50,0.500
23350,V1_998,V2_5400,0,1.0,1.0,1.00,1.0,1.0,1.0,1.0,...,7.36,6.37,6.38,6.39,6.4,6.41,6.42,0.0,1.00,0.500
23351,V1_998,V2_6886,0,1.0,1.0,1.00,1.0,1.0,1.0,1.0,...,7.36,7.37,7.38,7.39,7.4,7.41,7.42,1.0,1.00,1.000
23352,V1_999,V2_1605,1,1.0,1.0,0.25,1.0,0.5,0.5,1.0,...,6.36,6.37,6.38,6.39,6.4,6.41,6.42,0.0,0.50,0.125


In [80]:
ndf__ = ndf_.copy()
ndf__.drop(ndf_.iloc[:,3:547].columns, axis='columns', inplace=True)
ndf__

,StudentId,AssessmentId,FirstAssessment_byStudent,OrganizationId,IsMale,bit_multipleAssessors_grpStudent,medTime_startAssmnt_grpStudent,avgTime_startAssmnt_grpStudent,medTime_endAssmnt_grpStudent,avgTime_endAssmnt_grpStudent,...,Z21_,Z22_,Z23_,Z24_,Z25_,Z26_,Z27_,Z28_,Basic_Skills,Advanced_Skills
0,V1_0,V2_0,1,V548_0,1.0,0.0,2:59:19 PM,4:21:23 PM,3:45:43 PM,4:59:06 PM,...,6.36,6.37,6.38,6.39,6.4,6.41,6.42,0.0,0.00,0.000
1,V1_0,V2_122,0,V548_0,1.0,0.0,2:59:19 PM,4:21:23 PM,3:45:43 PM,4:59:06 PM,...,7.36,7.37,6.38,6.39,7.4,6.41,6.42,1.0,0.00,0.500
2,V1_1,V2_1,1,V548_1,1.0,1.0,5:42:40 PM,5:42:06 PM,10:28:53 PM,8:33:33 PM,...,6.36,6.37,6.38,6.39,6.4,6.41,6.42,0.0,0.00,0.000
3,V1_1,V2_572,0,V548_1,1.0,1.0,5:42:40 PM,5:42:06 PM,10:28:53 PM,8:33:33 PM,...,7.36,7.37,7.38,7.39,7.4,6.41,7.42,0.0,0.50,0.500
4,V1_10,V2_11,1,V548_4,1.0,1.0,6:21:11 PM,5:47:55 PM,7:16:02 PM,7:10:04 PM,...,6.36,6.37,6.38,6.39,6.4,6.41,6.42,0.0,0.25,0.500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23349,V1_998,V2_3546,0,V548_51,1.0,1.0,4:36:24 PM,10:36:42 AM,2:26:36 AM,9:10:33 AM,...,7.36,6.37,6.38,6.39,6.4,6.41,6.42,0.0,0.50,0.500
23350,V1_998,V2_5400,0,V548_51,1.0,1.0,4:36:24 PM,10:36:42 AM,2:26:36 AM,9:10:33 AM,...,7.36,6.37,6.38,6.39,6.4,6.41,6.42,0.0,1.00,0.500
23351,V1_998,V2_6886,0,V548_51,1.0,1.0,4:36:24 PM,10:36:42 AM,2:26:36 AM,9:10:33 AM,...,7.36,7.37,7.38,7.39,7.4,7.41,7.42,1.0,1.00,1.000
23352,V1_999,V2_1605,1,V548_62,0.0,0.0,5:22:08 PM,5:16:15 PM,6:13:40 PM,6:05:28 PM,...,6.36,6.37,6.38,6.39,6.4,6.41,6.42,0.0,0.50,0.125


In [81]:
ndf__.to_csv(data_path + "normalized_grouped_data_v1.csv")